# Preparing Training Data (remote sensing) for DeepLab

Training data is critical for supervised learning. There are many public training data available online, such as [ImageNet](http://www.image-net.org/) and [PASCAL VOC 2012](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/), can be used to train deep learning algorithms. Images in these datasets are from everyday scenes and contain targets such as cats, cars, and people.   

We need to prepare our new, unique training data because both images and targets are different from those in the public datasets. We use images from satellites or airborne platforms, and targets are landforms, landslides, and other objects on the surface. 

To understand how we prepare trianing data for DeepLab, we will go through each step in this tutorial. [DeepLab](https://github.com/tensorflow/models/tree/master/research/deeplab) is a supervised learning algorithm for labeling targets on images. An UAV-derived image covering Eboling Mountain on the Tibetan Plateau will be used for the demonstration. Please refer to [Huang et al., 2018](https://www.mdpi.com/2072-4292/10/12/2067) for more information.

## A glimpse on training data

#### Training images and the corresponding label images from PASCAL VOC 2012:
![title](figures/voc_air_plane_layout_trim.png ) 
In this example, the label image (pixel-wise class map) shows where are the targets (airple) and background (sky). Usually, the pixels in the label images store the class ID (an integer), which is assigned with a specific color (e.g., red) for visualization. These label images were created by humans using annotation tools such as [labelme](https://github.com/wkentaro/labelme).

#### Training polygons and remote sensing images:

 <img src=figures/training_polygon_source_trim.jpg width="800"/>
The figure above shows the distribution of the training polygons: 15 of them are non-gullies; seven were collected from field GPS measurements; and four were from manual delineation (digitizing).
 
Compared with images from everyday scenes, remote sensing images have characteristics of a large size (e.g., > 10000 by 10000 pixels), multiple bands (>= three), and coordinates. The coordinates are very important because they represent target locations on the earth's surface.

Two categories of training polygons: positive and negative polygons, are presented in the figure above. Positive ones are target boundaries (e.g., gully polygons), and negative ones are these covering non-target areas.

<!-- #### Training data derived from remote sensing images:  -->


## Step 1: delineating training polgyons

As mentioned in the previous cell, positive training polygons are target boundaries can be delineated on images or measured in the field. These postitive ones are also ground truhts used from validation of the mapping algorithm. If someone else already prepared these ground truths, then you can take them and save a lot of time. 

In the Eboling study, the ground truth polygons were collected using the Real-time Kinematic (RTK) measurements in the field and manually delineated (digitizing) on the UAV-derived images(aka DOM). 

The steps of delineation of polygons in [QGIS](https://www.qgis.org) include: (1) "New Shapefile Layer" and choosing "Type" as "Polygon"; (2) adding a new field name "class_int" to the shapefile; (3) "Add feature", i.e., adding polygons to the shapefile layer and inputitng "class id". 


We also need to add some negative training polygons because they help distinguish the gullies from the similar non-gully land covers during the training. One practical way to add negative ones and make them representative for training is that: running an initial mapping exercise by only using ground truths as training polygons, then adding negative ones to cover areas containing numerous false positives. Otherwise, creating negative training polygons requires expertise and ground knowledge of all the land covers in the study area.


## Step 2: generating training data similar to PASCAL VOC 2012 dataset

Because DeepLab only accept training data like PASCAL VOC dataset, we need to convert the training polygons and remote sensing images to this format. 

In [26]:
# for demonatration, the image was downsampled from 0.15 m to 0.48 m.
dom_path = "eboling_image_polygons/UAV_DOM_Eboling_0.48m.tif" # path of remote sensing images

# the training polygons, inlcuding 70% of ground truths and negativ polygons
training_polygon_path = "eboling_image_polygons/train_polygons_digitize_gps_v5.shp" # path of the training polygons

Make sure that the training data and remote sensing images have the same projection (Coordinate System)

In [27]:
# optional: output the prjection in proj4 format, and make sure they should be the same.
!gdalsrsinfo -o proj4 eboling_image_polygons/UAV_DOM_Eboling_0.48m.tif
!gdalsrsinfo -o proj4 eboling_image_polygons/train_polygons_digitize_gps_v5.shp

+proj=tmerc +lat_0=0 +lon_0=99 +k=1 +x_0=500000 +y_0=0 +datum=WGS84 +units=m +no_defs 
+proj=tmerc +lat_0=0 +lon_0=99 +k=1 +x_0=500000 +y_0=0 +datum=WGS84 +units=m +no_defs 


### Prepare a parameter file
Most of the input files and parameter are set in a parameter file (e.g., para_pre.ini). Please open para_pre.ini using vim or other text editor and set the parameter before running scripts.

Parameters the step of preparing training data include:
1. working_root
2. input_image_dir
3. input_image_rescale
4. training_polygons
5. training_polygons_sub
6. input_train_dir
7. input_label_dir
8. train_patch_width
9. train_patch_height
10. train_pixel_overlay_x
11. train_pixel_overlay_y




In [30]:
%env para_file=para_pre.ini
# set the code path
%env eo_dir=/Users/huanglingcao/codes/PycharmProjects/Landuse_DL  
!echo ${eo_dir}
# # remove previous data or results if necessary
!${eo_dir}/thawslumpScripts/remove_previous_data.sh ${para_file}


env: para_file=para_pre.ini
env: eo_dir=/Users/huanglingcao/codes/PycharmProjects/Landuse_DL
/Users/huanglingcao/codes/PycharmProjects/Landuse_DL
remove_previous_data.sh : remove previous data or results to run again


In [32]:
#extract sub_images based on the training polgyons
# !${eo_dir}/thawslumpScripts/get_sub_images_qtp.sh ${para_file}

get_sub_images_qtp.sh : extract sub-images and sub-labels for a given shape file (training polygons)
2019-10-27 11:38:31: gdalsrsinfo -o proj4 /Users/huanglingcao/codes/PycharmProjects/Landuse_DL/tutorial/eboling_image_polygons/train_polygons_digitize_gps_v5.shp 
2019-10-27 11:38:31: gdalsrsinfo -o proj4 /Users/huanglingcao/codes/PycharmProjects/Landuse_DL/tutorial/eboling_image_polygons/train_polygons_digitize_gps_v5.shp 
2019-10-27 11:38:32: gdalsrsinfo -o proj4 /Users/huanglingcao/codes/PycharmProjects/Landuse_DL/tutorial/eboling_image_polygons/train_polygons_digitize_gps_v5.shp 
2019-10-27 11:38:32: gdalsrsinfo -o proj4 /Users/huanglingcao/codes/PycharmProjects/Landuse_DL/tutorial/eboling_image_polygons/UAV_DOM_Eboling_0.48m.tif 
2019-10-27 11:38:32: gdalsrsinfo -o proj4 /Users/huanglingcao/codes/PycharmProjects/Landuse_DL/tutorial/eboling_image_polygons/train_polygons_digitize_gps_v5.shp 
2019-10-27 11:38:32: obtaining 0th sub-image and the corresponding label raster
2019-10-27 11

### A invalid polygon
The error above may also occur when you obtaining sub-images. The error reported a self-intersection at point '668045.59660145082 4209336.0392965544'

In QGIS, zoom in to this point using a tool named [zoomtocoordinates](https://plugins.qgis.org/plugins/zoomtocoordinates/), then we can see the self-intersection as the figure below:
<img src=figures/self-intersection_example.png width="500" />

Then fix this issue by eidting the polygon and remove the self-intersection. After fixing the error, we run the script again.


In [33]:
#extract sub_images based on the training polgyons
!${eo_dir}/thawslumpScripts/get_sub_images_qtp.sh ${para_file}

get_sub_images_qtp.sh : extract sub-images and sub-labels for a given shape file (training polygons)
2019-10-27 11:53:22: gdalsrsinfo -o proj4 /Users/huanglingcao/codes/PycharmProjects/Landuse_DL/tutorial/eboling_image_polygons/train_polygons_digitize_gps_v5.shp 
2019-10-27 11:53:23: gdalsrsinfo -o proj4 /Users/huanglingcao/codes/PycharmProjects/Landuse_DL/tutorial/eboling_image_polygons/train_polygons_digitize_gps_v5.shp 
2019-10-27 11:53:23: gdalsrsinfo -o proj4 /Users/huanglingcao/codes/PycharmProjects/Landuse_DL/tutorial/eboling_image_polygons/train_polygons_digitize_gps_v5.shp 
2019-10-27 11:53:23: gdalsrsinfo -o proj4 /Users/huanglingcao/codes/PycharmProjects/Landuse_DL/tutorial/eboling_image_polygons/UAV_DOM_Eboling_0.48m.tif 
2019-10-27 11:53:23: gdalsrsinfo -o proj4 /Users/huanglingcao/codes/PycharmProjects/Landuse_DL/tutorial/eboling_image_polygons/train_polygons_digitize_gps_v5.shp 
2019-10-27 11:53:24: obtaining 0th sub-image and the corresponding label raster
2019-10-27 11

### check sub-images the corresponding label raster

QGIS is the best way for visualiaztion, as follows, 
<img src=figures/qgis_label_raster.png width="600" />

The coordinates, pixel values, and training polygons can be checked together in QGIS.



In [37]:
# split sub-images to patches
!${eo_dir}/thawslumpScripts/split_sub_images.sh ${para_file}

split_sub_images.sh : split sub-images and sub-labels
input Width 1938  Height 1817
(0, 0, 471, 471)
Input file size is 1938, 1817
0...10...20...30...40...50...60...70...80...90...100 - done.
(0, 171, 471, 621)
Input file size is 1938, 1817
0...10...20...30...40...50...60...70...80...90...100 - done.
(0, 492, 471, 621)
Input file size is 1938, 1817
0...10...20...30...40...50...60...70...80...90...100 - done.
(0, 813, 471, 621)
Input file size is 1938, 1817
0...10...20...30...40...50...60...70...80...90...100 - done.
(0, 1134, 471, 621)
Input file size is 1938, 1817
0...10...20...30...40...50...60...70...80...90...100 - done.
(0, 1455, 471, 362)
Input file size is 1938, 1817
0...10...20...30...40...50...60...70...80...90...100 - done.
(171, 0, 621, 471)
Input file size is 1938, 1817
0...10...20...30...40...50...60...70...80...90...100 - done.
(171, 171, 621, 621)
Input file size is 1938, 1817
0...10...20...30...40...50...60...70...80...90...100 - done.
(171, 492, 621, 621)
Input file si

After subdividing, there are many image and label patches in folder *split_images* and *split_labels*.

## Data augmentation

To increase to volume and diversity of training data, we perform data agumentation for the image patch, the parameters for data augmentation in "para_pre.ini" file include:
1. data_augmentation
2. data_aug_ignore_classes

To perform data augmentation, we run the following script:


In [38]:
# data augmentation 
!${eo_dir}/thawslumpScripts/training_img_augment.sh ${para_file}

training_img_augment.sh : Perform image augmentation
image format:  .png
image augmentation on image patches
Augmentation of image (1 / 608)
Augmentation of image (2 / 608)
Augmentation of image (3 / 608)
Augmentation of image (4 / 608)
Augmentation of image (5 / 608)
Augmentation of image (6 / 608)
Augmentation of image (7 / 608)
Augmentation of image (8 / 608)
Augmentation of image (9 / 608)
Augmentation of image (10 / 608)
Augmentation of image (11 / 608)
Augmentation of image (12 / 608)
Augmentation of image (13 / 608)
Augmentation of image (14 / 608)
Augmentation of image (15 / 608)
Augmentation of image (16 / 608)
Augmentation of image (17 / 608)
Augmentation of image (18 / 608)
Augmentation of image (19 / 608)
Augmentation of image (20 / 608)
Augmentation of image (21 / 608)
Augmentation of image (22 / 608)
Augmentation of image (23 / 608)
Augmentation of image (24 / 608)
Augmentation of image (25 / 608)
/Users/huanglingcao/programs/anaconda3/lib/python3.6/site-packages/skimage/

We also can visualize these patches in QGIS. Notice that, the patches saved by data augmentation script lost their coordinates. It's ok because they are only for the purpose of training, which does not require coordinates. 

## Summary

1. We go through the steps of preparation of training data. 
2. Jupyter notebook is good for demonstration and explanation. However, when I run this script, I would like to put all the script into one bash file, then run the bash file.

### Next
1. Traning, which requires Graphics processing units (GPUs), otherwise it will take too long. 